# Active Inference cadCAD model

This notebook explores multi-agent active inference simulations by representing agents and their locations in a dictionary format..

## cadCAD Standard Notebook Layout

### 0. Dependencies

In [1]:
import pandas as pd
import sys

# adding tools to the system path
sys.path.insert(0, '../')

from radcad import Model, Simulation, Experiment

from tools.model import ActiveGridference

from blockference import gridference

# local utils
import tools.utils as u
from tools.control import construct_policies
import random as rand
from pymdp.maths import softmax

### Initializing Agent Network

In [2]:
grid = gridference.grid(10)
g_agent = gridference.GridAgent(grid)
agents = gridference.init_agents(5, grid)

### 1. State Variables

In [3]:
initial_state = {
    'agents': agents
}

### 2. System Parameters

In [4]:
params = {
}

### 3. Policy Functions

- `get_observation`
- `infer_states`
- `calc_efe`
- `calc_action_posterior`
- `sample_action`
- `calc_next_prior`
- `update_env_state`

In [5]:
def p_actinf(params, substep, state_history, previous_state):
    # State Variables
    agents = previous_state['agents']
    return gridference.actinf_dict(agents, g_agent)

### 4. State Update Functions

In [6]:
def s_agents(params, substep, state_history, previous_state, policy_input):

    agents_new = previous_state['agents'].copy()

    agent_updates = policy_input['agent_updates']

    if agent_updates != []:
        for update in agent_updates:
            s = update['source']
            agent = agents_new[s]
            update_prior = update['update_prior']
            update_env = update['update_env']
            update_action = update['update_action']
            update_inference = update['update_inference']

            agent.prior = update_prior
            agent.env_state = update_env
            agent.current_action = update_action
            agent.current_inference = update_inference

    return 'agents', agents_new

### 5. Partial State Update Blocks

In [7]:
state_update_blocks = [
    {
        'policies': {
            'p_actinf': p_actinf
        },
        'variables': {
            'agents': s_agents
        }
    }
]

### 6. Configuration

In [8]:
model = Model(
    # Model initial state
    initial_state=initial_state,
    # Model Partial State Update Blocks
    state_update_blocks=state_update_blocks,
    # System Parameters
    params=params
)

### 7. Execution

In [9]:
simulation = Simulation(
    model=model,
    timesteps=100,  # Number of timesteps
    runs=1  # Number of Monte Carlo Runs
)

In [10]:
result = simulation.run()

### 8. Analysis

In [11]:
pd.options.plotting.backend = "plotly"

In [12]:
df = pd.DataFrame(result)
df

,agents,simulation,subset,run,substep,timestep
0,{0: <tools.model.ActiveGridference object at 0...,0,0,1,0,0
1,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,1
2,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,2
3,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,3
4,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,4
...,...,...,...,...,...,...
96,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,96
97,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,97
98,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,98
99,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,99


In [13]:
df['agents'][100]

{0: <tools.model.ActiveGridference at 0x1509c2160>,
 1: <tools.model.ActiveGridference at 0x1509c23d0>,
 2: <tools.model.ActiveGridference at 0x1509c2640>,
 3: <tools.model.ActiveGridference at 0x1509c28b0>,
 4: <tools.model.ActiveGridference at 0x1509c2b20>}

In [14]:
df_unpacked = pd.DataFrame()
for d in df['agents']:
    for i, agent in d.items():
        df_unpacked = df_unpacked.append({f'{i}_A': agent.A, f'{i}_B': agent.B, f'{i}_c': agent.C, f'{i}_prior': agent.prior, f'{i}_env_state': agent.env_state}, ignore_index=True)

/var/folders/xj/yxwtvrv95n77ycc9hpngfr700000gn/T/ipykernel_44699/82352992.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_unpacked = df_unpacked.append({f'{i}_A': agent.A, f'{i}_B': agent.B, f'{i}_c': agent.C, f'{i}_prior': agent.prior, f'{i}_env_state': agent.env_state}, ignore_index=True)
/var/folders/xj/yxwtvrv95n77ycc9hpngfr700000gn/T/ipykernel_44699/82352992.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_unpacked = df_unpacked.append({f'{i}_A': agent.A, f'{i}_B': agent.B, f'{i}_c': agent.C, f'{i}_prior': agent.prior, f'{i}_env_state': agent.env_state}, ignore_index=True)
/var/folders/xj/yxwtvrv95n77ycc9hpngfr700000gn/T/ipykernel_44699/82352992.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_unpack

In [29]:
df_unpacked

,0_A,0_B,0_c,0_prior,0_env_state,1_A,1_B,1_c,1_env_state,1_prior,...,8_A,8_B,8_c,8_env_state,8_prior,9_A,9_B,9_c,9_env_state,9_prior
0,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[1.0, 0.0, 1.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0, 0)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[1.0, 0.0, 1.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0, 0)","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[1.0, 0.0, 1.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(3, 0)","[9.999999999999931e-33, 9.999999999999931e-33,...",NaN,NaN,NaN,NaN,NaN
